In [11]:
!pip install pytesseract


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
#final chatbot
import panel as pn
from langchain_community.llms import Ollama

# Load the Ollama language model
llm = Ollama(model="Finance_l2")

# Create tabs for different sections of the dashboard
text_input = pn.widgets.TextInput(name="Text Input", placeholder="Enter your text here")
invoke_button = pn.widgets.Button(name="Invoke Model", button_type="primary")
continue_button = pn.widgets.Button(name="Continue")
exit_button = pn.widgets.Button(name="Exit")
output_text = pn.pane.Markdown("", width=800)
prompt_history = []

def update_output_text():
    output_text.object = "\n\n".join(prompt_history)

def invoke_model(event):
    global prompt_history
    input_text = text_input.value
    output = llm.invoke(input_text)
    prompt_history.append(f"Prompt: {input_text}\n\n*Answer:* {output}")
    update_output_text()
    text_input.disabled = True
    invoke_button.disabled = True
    continue_button.disabled = False
    exit_button.disabled = False

def continue_prompt(event):
    text_input.disabled = False
    invoke_button.disabled = False
    continue_button.disabled = True
    exit_button.disabled = True

def exit_prompt(event):
    global prompt_history
    prompt_history = []
    update_output_text()
    continue_prompt(event)

invoke_button.on_click(invoke_model)
continue_button.on_click(continue_prompt)
exit_button.on_click(exit_prompt)

# Create tabs for different sections of the dashboard
tab1_content = pn.Column(
    pn.pane.Markdown("## Hi, I am your personal Financial assistant! "),
    pn.pane.Markdown("### Enter your prompt here "),
    text_input,
    invoke_button,
    output_text,
    pn.Row(continue_button, exit_button)
)

# Create the main layout for the Ollama language model section
ollama_main_layout = pn.Row(
    pn.Spacer(width=200),  # Spacer to center-align content
    pn.Column(tab1_content, align="center"),
    pn.Spacer(width=200),  # Spacer to center-align content
)

# Create the sidebar for the Ollama language model section
ollama_sidebar = pn.Column(
    pn.pane.Markdown("# Ollama Language Model"),
    pn.pane.Markdown("This dashboard allows you to interact with the Ollama language model."),
    pn.pane.PNG("ollama.png", sizing_mode="scale_both")
)

# Create the dashboard template for the Ollama language model section
ollama_template = pn.template.FastListTemplate(
    title='Financial Planner',
    header_background="#183a1d",
    sidebar=ollama_sidebar,
    main=ollama_main_layout
)

# Show the Ollama language model dashboard
ollama_template.show()

Launching server at http://localhost:50380


In [49]:
#final for ocr
import panel as pn
from langchain_community.llms import Ollama
from PIL import Image
import io
import pytesseract

# Load the Ollama language model
llm = Ollama(model="Finance_l2")

# Set the path to the Tesseract executable (update this with your installation path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# Create tabs for different sections of the dashboard
tab1_content = pn.Column(
    pn.pane.Markdown("## Hi, I am your personal Financial assistant! "),
    pn.widgets.FileInput(accept=".png, .jpg, .jpeg", name="Upload Image"),
    pn.widgets.Button(name="Perform OCR & Invoke Model", button_type="primary"),
    pn.pane.Str(name="Output:", width=800)  # Pane to display the output
)

# Create the main layout for the Ollama language model section
ollama_main_layout = pn.Row(
    pn.Spacer(width=200),  # Spacer to center-align content
    pn.Column(tab1_content, align="center"),
    pn.Spacer(width=200),  # Spacer to center-align content
)

# Create the dashboard template for the Ollama language model section
ollama_template = pn.template.FastListTemplate(
    title='Financial Planner',
    header_background="#183a1d",
    main=ollama_main_layout
)

# Define a callback to perform OCR and invoke the model
def perform_ocr_and_invoke_model(event):
    # Get the uploaded image file
    uploaded_image = tab1_content[1].value
    
    if uploaded_image is not None:
        # Read the image data
        image_data = uploaded_image
        
        # Open the image using PIL
        img = Image.open(io.BytesIO(image_data))
        
        # Perform OCR on the image
        ocr_result = pytesseract.image_to_string(img)
        
        # Invoke the model with the OCR result
        output_text = llm.invoke(ocr_result)
        
        # Update the output pane with the model output
        tab1_content[-1] = pn.pane.Str(output_text, width=800)
    else:
        tab1_content[-1] = pn.pane.Str("Please upload an image first.", width=800)

# Assign the callback to the button
tab1_content[2].on_click(perform_ocr_and_invoke_model)

# Show the Ollama language model dashboard
ollama_template.show()



Launching server at http://localhost:50409


In [48]:
#piechart bargraph
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])

# Display categorized data table
categorized_table = pn.widgets.DataFrame(df)

def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )

# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    main=[pn.Row(
        categorized_table,
        tabs
    )]
)

template.show()

Launching server at http://localhost:50392
